# Боты в телеге

## 1 Проверяет доступность к сайту

In [1]:
pip install pytelegrambotapi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import telebot
from urllib.parse import urlparse
from telebot import types
import requests

bot = telebot.TeleBot('')

def check_site(url):
    parsed = urlparse(url,scheme='http')
    url = f'''{parsed.scheme}://{parsed.netloc if parsed.netloc!='' else 'www'}.{parsed.path}#{parsed.fragment}'''
    print(url)
    response = requests.get(url)
    if(response.status_code == 200):
        return True
    return False

@bot.message_handler(content_types = ["text"])
def handle_text(message):
    text:str = message.text
    if (text.startswith('/')):
        if (text.startswith('/start')):
            bot.send_message(message.from_user.id, text = 'Введите ссылку для проверки доступности сайта')
        else:
            bot.send_message(message.from_user.id, "Неизвестная команда")
    else:
        if(check_site(text)):
            bot.send_message(message.from_user.id, f'Сайт по ссылке: {text}  доступен')
        else:
            bot.send_message(message.from_user.id, f'Сайт по ссылке: {text}  недоступен')

@bot.message_handler(commands = ["start"])
def start(k, res = False):
    print(k)
    bot.send_message(k.chat.id, 'Привет')
bot.infinity_polling()

2023-12-12 18:42:22,000 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-12-12 18:42:22,001 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


## 2 Анализатор текста

In [ ]:
import telebot
bot = telebot.TeleBot('')

def create_stats(text):
    w = text.split()
    def replace_all(word,what):
        for l in what:
            word=word.replace(l,"")
        return word
    w = [replace_all(word.lower(),['.','!','?',',','-']) for word in w]
    c = dict()
    for word in w:
        c[word] = c[word]+1 if word in c else 1
    max_len = 0
    word_max_len = None
    for word,_ in c.items():
        if(len(word)>max_len):
            max_len = len(word)
            word_max_len = word
    max_cnt = 0
    word_max_cnt = None
    for word,cnt in c.items():
        if ( cnt > max_cnt ):
            word_max_cnt = word
            max_cnt = cnt
    return f"Самое длинное слово в тексте: {word_max_len}, чаще всего встречается в тексте: {word_max_cnt}"

@bot.message_handler(content_types=["text"])
def handle_text(message):
    text:str = message.text
    if (text.startswith('/')):
        if (text.startswith('/start')):
            bot.send_message(message.from_user.id, "Здарова бандит!")
        else:
            bot.send_message(message.from_user.id, "Неизвестная команда")
    else:
        stats = create_stats(text)
        bot.send_message(message.from_user.id, stats)
@bot.message_handler(commands=["start"])
def start(k, res = False):
    print(k)
    bot.send_message(k.chat.id, 'Hello!')
bot.infinity_polling()

2023-12-12 18:42:30,933 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-12-12 18:42:30,934 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


## 3 бот калькулятор

In [8]:
pip install python-telegram-bot

                                              0.0/552.6 kB ? eta -:--:--
     --                                      30.7/552.6 kB 1.4 MB/s eta 0:00:01
     --                                    41.0/552.6 kB 393.8 kB/s eta 0:00:02
     -------                              112.6/552.6 kB 939.4 kB/s eta 0:00:01
     -------                              112.6/552.6 kB 939.4 kB/s eta 0:00:01
     -------------------                    276.5/552.6 kB 1.4 MB/s eta 0:00:01
     ---------------------                  307.2/552.6 kB 1.4 MB/s eta 0:00:01
     --------------------------------       471.0/552.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------  542.7/552.6 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 552.6/552.6 kB 1.7 MB/s eta 0:00:00
                                              0.0/75.0 kB ? eta -:--:--
     ---------------------------------------- 75.0/75.0 kB 2.1 MB/s eta 0:00:00
                                              0.0/83.9 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import telebot, math, time

bot = telebot.TeleBot('', parse_mode='html')

value = ''
old_value = ''
keybord = telebot.types.InlineKeyboardMarkup()
keybord.row( telebot.types.InlineKeyboardButton('🧹', callback_data='c'),
             telebot.types.InlineKeyboardButton('◀️', callback_data='<='),
             telebot.types.InlineKeyboardButton('o/a', callback_data='oa'),
             telebot.types.InlineKeyboardButton('o/g', callback_data='og'),
             telebot.types.InlineKeyboardButton('➗', callback_data='/') )

keybord.row( telebot.types.InlineKeyboardButton('7️⃣', callback_data='7'), 
             telebot.types.InlineKeyboardButton('8️⃣', callback_data='8'),
             telebot.types.InlineKeyboardButton('9️⃣', callback_data='9'),
             telebot.types.InlineKeyboardButton('✖️', callback_data='*') )

keybord.row( telebot.types.InlineKeyboardButton('4️⃣', callback_data='4'), 
             telebot.types.InlineKeyboardButton('5️⃣', callback_data='5'),
             telebot.types.InlineKeyboardButton('6️⃣', callback_data='6'),
             telebot.types.InlineKeyboardButton('➖', callback_data='-') )

keybord.row( telebot.types.InlineKeyboardButton('1️⃣', callback_data='1'), 
             telebot.types.InlineKeyboardButton('2️⃣', callback_data='2'),
             telebot.types.InlineKeyboardButton('3️⃣', callback_data='3'),
             telebot.types.InlineKeyboardButton('➕', callback_data='+') )

keybord.row( telebot.types.InlineKeyboardButton('+/-', callback_data='pm'), 
             telebot.types.InlineKeyboardButton('0️⃣', callback_data='0'),
             telebot.types.InlineKeyboardButton(',', callback_data=','),
             telebot.types.InlineKeyboardButton('🟰', callback_data='=') )

@bot.message_handler(commands=['start', 'calc'])
def getMessage(message):
    if(message.text == '/start'):
        bot.reply_to(message, "Hi 👋🏻 This is a calculator bot for Telegram\n\n👉🏻 Send /calc command to start the calculator!")
    elif (message.text == '/calc'):
        global value
        if value == '':
            bot.send_message(message.from_user.id, "0", reply_markup=keybord)
        else:
            bot.send_message(message.from_user.id, value, reply_markup=keybord)

@bot.callback_query_handler(func=lambda call: True)
def callback_func(query):
    global value, old_value
    data = query.data
    if data == 'no':
        pass
    elif data == 'oa':
        list  = value.strip(',').split(',')
        res = ''
        for index in list:
            res += index + "+"

        value = str(eval(res.strip('+')) / len(list))
    elif data == 'og':
        list  = value.strip(',').split(',')
        res = ''
        for index in list:
            res += index + "*"

        value = str(math.pow(eval(res.strip('*')), (1 / len(list))))
    elif data == 'c':
        value = ''
    elif data == '<=':
        if value != '':
            value = value[:len(value) - 1]
    elif data == 'pm':
        value = str(-1 * eval(value))
    elif data == '=':
        try:
            value = str(eval(value))
        except:
            value = 'Error!' 
    else:
        value += data

    if ( value != old_value and value != '' ) or ('0' != old_value and value == '' ):
        if value == '':
            bot.edit_message_text(chat_id=query.message.chat.id, message_id=query.message.message_id, text='0', reply_markup=keybord)
            old_value = '0'
        else: 
            bot.edit_message_text(chat_id=query.message.chat.id, message_id=query.message.message_id, text=value, reply_markup=keybord)
            old_value = value
    
    if value == 'Error!' : value = '0'

while True:
    try:
        belgi = True
        bot.polling(none_stop=True)
    except Exception as e:
        if belgi == True:
            bot.send_message('admin telegram user id', f'<b>Server Error ❗️</b>\n\n<b>Content:</b>\n<code>{e}</code>')
            belgi = False
        time.sleep(5)